In [1]:
!wget  -O 'Bitcoin_kaggle.csv' -q https://www.dropbox.com/scl/fi/p58p467ueqcjldk9f798y/Bitcoin_kaggle.csv?rlkey=jl7ki4s17kmvbjhzhii8oku61&dl=0
!wget  -O 'prediction_baseline.csv' -q https://www.dropbox.com/scl/fi/y2m3z2k2ge0mh6li5557e/prediction_baseline.csv?rlkey=hssa6l35e0h4xbtxek6nlg0pt&dl=0

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime

In [5]:
df = pd.read_csv('Bitcoin_kaggle.csv', sep=';')
df = df[::-1]
prediction = pd.read_csv('prediction_baseline.csv', sep=',')

In [6]:
def transform_numeric_columns(df, columns):
    for col in columns:
        df[col] = df[col].str.replace(',', '').astype(float)
    return df

df = transform_numeric_columns(df, ['Open', 'Price', 'High', 'Low'])

In [7]:
def convert_volume(volume_str):
    if volume_str.endswith('K'):
        return float(volume_str[:-1]) * 1000
    if volume_str.endswith('M'):
        return float(volume_str[:-1]) * 1000000
    if volume_str.endswith('B'):
        return float(volume_str[:-1]) * 1000000000
    return float(volume_str)

df['Vol.'] = df['Vol.'].apply(convert_volume)

In [8]:
def convert_percent(percent_str):
    return float(percent_str.rstrip('%'))

In [9]:
df['Change %'] = df['Change %'].apply(convert_percent)

In [10]:
for i in range(1, 7):
    df[f'lag_{i}'] = df['Open'].shift(i)

df = df.dropna()

In [11]:
train = df[:-7]
test = df[-7:]

In [12]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(train[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'High', 'Low', 'Vol.', 'Change %']], train['Price'])

forecast = model.predict(test[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'High', 'Low', 'Vol.', 'Change %']])
test['Date'] = test['Date']
test['Price'] = forecast

test['Date'] = pd.to_datetime(test['Date'], format='%b %d, %Y').dt.strftime('%Y-%m-%d')

result = test[['Date', 'Price']]
result.to_csv('bitcoin_forecast.csv', index=False)

<ipython-input-12-a91de85bf77b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Date'] = test['Date']
<ipython-input-12-a91de85bf77b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Price'] = forecast
<ipython-input-12-a91de85bf77b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retur

In [4]:
see = pd.read_csv('bitcoin_forecast.csv',  sep=',')
see

,Date,Price
0,2023-07-08,30333.261
1,2023-07-09,30374.484
2,2023-07-10,30366.684
3,2023-07-11,30347.516
4,2023-07-12,30344.307
5,2023-07-13,30320.723
6,2023-07-14,30320.723
